In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
from konlpy.tag import Okt
from tqdm import tqdm

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 35.2 MB/s eta 0:00:00


In [ ]:
# 리뷰 데이터 로드 (Google Colab에 파일을 업로드해야 합니다)
from google.colab import files
uploaded = files.upload()

Saving each_reviews.csv to each_reviews.csv


In [ ]:
# 파일명은 각자 파일명으로 변경
file_path = 'each_reviews.csv'
reviews_df = pd.read_csv(file_path)

In [ ]:
reviews_df = reviews_df[:10000][:]

In [ ]:
reviews_df

,사용자,가게명,리뷰
0,서귀포오,쿠쿠아림,직원인지 사장인지 굉장히 일하기 싫은듯. 처음 입장부터 나갈때까지 인상 쓰고 있고 ...
1,서귀포오,과수원민박,..\n중문에서 유명한 콩국수 맛집\n기본간이 살짝 달달해서 설탕파들이 좋아할 맛....
2,서귀포오,녹차골정식,..\n가격대비 구성은 나쁘지 않음.\n날벌레 많은데 반찬통 열어놓아서 별로.\n저...
3,서귀포오,서귀포밀면,..\n밀면 땡길때 가끔 가는 곳.
4,서귀포오,엉또정 서귀포본점,..\n김치전골보다 두루치기가 더 나은듯.\n반찬수는 적음.\n주차 편함.
...,...,...,...
199770,달꼬매,린스테이블,가게도 깨끗하고 포장 설명도 꼼꼼하고 친절하게\n해주셨어요
199771,달꼬매,대호축산,사장님 친절하세요\n부위별로 따로 요청하면 그 자리에서 바로 포장가능합니다 택배도 ...
199772,달꼬매,한입소반,김밥 매장이라기보다 약간 공장같은 느낌이지만 깔끔하고 김밥도 너무 맛있네요 김밥 하...
199773,달꼬매,몽트비어,신랑이 술을 좋아해서 찾아간 수제 맥주 공장! 종류별로 병맥 골라봤습니다!


In [ ]:
# NaN 값을 빈 문자열로 대체
reviews_df['리뷰'] = reviews_df['리뷰'].fillna('')

# 한국어 불용어 리스트 (예시, 필요에 따라 확장 가능)
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '를', '으로', '자', '에', '와', '한', '하다', '고', '에서', '이다', '다']


In [ ]:
def preprocess_text(text, remove_newlines=False):
    # 개행문자 제거 여부 설정
    if remove_newlines:
        text = text.replace('\n', ' ')

    # 구두점 및 특수 문자 제거
    text = re.sub(r'[^\w\s]', '', text)

    # 형태소 분석 및 불용어 제거
    okt = Okt()
    tokens = okt.morphs(text)
    tokens = [word for word in tokens if word not in stopwords]

    return ' '.join(tokens)

In [ ]:
# 리뷰 텍스트 전처리
remove_newlines = True  # 개행문자 제거 여부 설정
reviews_df['processed_review'] = reviews_df['리뷰'].apply(lambda x: preprocess_text(x, remove_newlines))

In [ ]:
reviews_df

,사용자,가게명,리뷰,processed_review
0,서귀포오,쿠쿠아림,직원인지 사장인지 굉장히 일하기 싫은듯. 처음 입장부터 나갈때까지 인상 쓰고 있고 ...,직원 인지 사장 인지 굉장히 일 하기 싫은듯 처음 입장 부터 나갈 때 까지 인상 쓰...
1,서귀포오,과수원민박,..\n중문에서 유명한 콩국수 맛집\n기본간이 살짝 달달해서 설탕파들이 좋아할 맛....,중문 유명한 콩국수 맛집 기본 간 살짝 달달 해서 설탕 파 좋아할 맛 마시는 물 냉...
2,서귀포오,녹차골정식,..\n가격대비 구성은 나쁘지 않음.\n날벌레 많은데 반찬통 열어놓아서 별로.\n저...,가격 대비 구성은 나쁘지 않음 날 벌레 많은데 반찬 통 열어놓아서 별로 저 자동차 ...
3,서귀포오,서귀포밀면,..\n밀면 땡길때 가끔 가는 곳.,밀면 땡길 때 가끔 가는 곳
4,서귀포오,엉또정 서귀포본점,..\n김치전골보다 두루치기가 더 나은듯.\n반찬수는 적음.\n주차 편함.,김치 전골 보다 두루치기 더 나은듯 반찬 수 적음 주차 편함
...,...,...,...,...
199770,달꼬매,린스테이블,가게도 깨끗하고 포장 설명도 꼼꼼하고 친절하게\n해주셨어요,가게 도 깨끗하고 포장 설명 도 꼼꼼하고 친절하게 해주셨어요
199771,달꼬매,대호축산,사장님 친절하세요\n부위별로 따로 요청하면 그 자리에서 바로 포장가능합니다 택배도 ...,사장 님 친절하세요 부위 별로 따로 요청 하면 그 자리 바로 포장 가능합니다 택배 ...
199772,달꼬매,한입소반,김밥 매장이라기보다 약간 공장같은 느낌이지만 깔끔하고 김밥도 너무 맛있네요 김밥 하...,김밥 매장 이라기 보다 약간 공장 같은 느낌 이지만 깔끔하고 김밥 도 너무 맛있네요...
199773,달꼬매,몽트비어,신랑이 술을 좋아해서 찾아간 수제 맥주 공장! 종류별로 병맥 골라봤습니다!,신랑 술 을 좋아해서 찾아간 수제 맥주 공장 종류 별로 병맥 골라 봤습니다


In [ ]:
# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# KoBERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert').to(device)

In [ ]:
# KoBERT 입력 형식으로 변환하는 함수
def bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128).to(device)
    outputs = model(**inputs)
    # CLS 토큰의 벡터를 사용
    return outputs.last_hidden_state[:, 0, :].detach().cpu().numpy()

In [ ]:
# 사용자별 리뷰 유사도 계산
user_similarity = {}

total_users = len(reviews_df['사용자'].unique())
with tqdm(total=total_users, desc="Processing users") as pbar:
    for user, group in reviews_df.groupby('사용자'):
        if len(group) > 1:  # 리뷰가 2개 이상인 사용자만 분석
            embeddings = []
            for review in group['processed_review']:
                embeddings.append(bert_embedding(review))
            embeddings = np.vstack(embeddings)
            cosine_similarities = cosine_similarity(embeddings)
            # 유사도 행렬의 상삼각행렬에서 주대각선을 제외한 값들의 평균을 계산
            mean_similarity = cosine_similarities[np.triu_indices_from(cosine_similarities, k=1)].mean()
            user_similarity[user] = mean_similarity
        pbar.update(1)



Processing users: 100%|██████████| 2979/2979 [35:49<00:00,  1.39it/s]


In [ ]:
sorted_user_similarity = sorted(user_similarity.items(), key=lambda item: item[1], reverse=True)

In [ ]:
sorted_user_similarity2 = sorted(user_similarity.items(), key=lambda item: item[1], reverse=True)

In [ ]:
for user, similarity in sorted_user_similarity:
    print(f"User: {user}, Average Similarity: {similarity:.2f}")

User: 양정호3, Average Similarity: 1.00
User: ownkayoung, Average Similarity: 1.00
User: 크리스미, Average Similarity: 0.97
User: jksoon, Average Similarity: 0.97
User: jju****, Average Similarity: 0.97
User: Vero79, Average Similarity: 0.94
User: epd****, Average Similarity: 0.93
User: 갬289, Average Similarity: 0.92
User: com4001, Average Similarity: 0.92
User: 시호만세, Average Similarity: 0.91
User: ryo****, Average Similarity: 0.90
User: 냥순이52, Average Similarity: 0.90
User: dwd****, Average Similarity: 0.90
User: 찐맛집다내꺼, Average Similarity: 0.90
User: wjd****, Average Similarity: 0.89
User: 보라공주0609, Average Similarity: 0.89
User: 뚱땅박하, Average Similarity: 0.89
User: 도문대작, Average Similarity: 0.89
User: shsh4886, Average Similarity: 0.89
User: 흰둥이흰둥이이, Average Similarity: 0.89
User: chh****, Average Similarity: 0.89
User: ywm88, Average Similarity: 0.89
User: 꿀꿀그램, Average Similarity: 0.88
User: 망글동이, Average Similarity: 0.88
User: 욤쿵, Average Similarity: 0.88
User: dlgpfus772, Average Simil

In [ ]:
for user, similarity in sorted_user_similarity2:
    print(f"User: {user}, Average Similarity: {similarity:.2f}")

User: 양정호3, Average Similarity: 1.00
User: ownkayoung, Average Similarity: 1.00
User: 크리스미, Average Similarity: 0.97
User: jksoon, Average Similarity: 0.97
User: jju****, Average Similarity: 0.97
User: Vero79, Average Similarity: 0.94
User: epd****, Average Similarity: 0.93
User: 갬289, Average Similarity: 0.92
User: com4001, Average Similarity: 0.92
User: 시호만세, Average Similarity: 0.91
User: ryo****, Average Similarity: 0.90
User: 냥순이52, Average Similarity: 0.90
User: dwd****, Average Similarity: 0.90
User: 찐맛집다내꺼, Average Similarity: 0.90
User: wjd****, Average Similarity: 0.89
User: 보라공주0609, Average Similarity: 0.89
User: 뚱땅박하, Average Similarity: 0.89
User: 도문대작, Average Similarity: 0.89
User: shsh4886, Average Similarity: 0.89
User: 흰둥이흰둥이이, Average Similarity: 0.89
User: chh****, Average Similarity: 0.89
User: ywm88, Average Similarity: 0.89
User: 꿀꿀그램, Average Similarity: 0.88
User: 망글동이, Average Similarity: 0.88
User: 욤쿵, Average Similarity: 0.88
User: dlgpfus772, Average Simil

In [ ]:
sorted_total_user_similarity = sorted(user_similarity.items(), key=lambda item: item[1], reverse=True)

In [ ]:
for user, similarity in sorted_total_user_similarity:
    print(f"User: {user}, Average Similarity: {similarity:.2f}")

User: 양정호3, Average Similarity: 1.00
User: kkomi222, Average Similarity: 1.00
User: 달콩s2, Average Similarity: 1.00
User: 쿠블cooble, Average Similarity: 1.00
User: ceh****, Average Similarity: 1.00
User: hhhhkhy, Average Similarity: 1.00
User: pink00ss, Average Similarity: 1.00
User: ps1****, Average Similarity: 1.00
User: 시인 김윤아, Average Similarity: 1.00
User: 이6754, Average Similarity: 1.00
User: soyonemy, Average Similarity: 1.00
User: 빼미27, Average Similarity: 1.00
User: ownkayoung, Average Similarity: 1.00
User: 도도맘35, Average Similarity: 1.00
User: 랄라3970, Average Similarity: 1.00
User: 비니3464, Average Similarity: 1.00
User: 짝꿍땡이, Average Similarity: 1.00
User: hsk****, Average Similarity: 1.00
User: michel423, Average Similarity: 1.00
User: son****, Average Similarity: 1.00
User: younmi1102, Average Similarity: 1.00
User: 봄날의 소풍, Average Similarity: 1.00
User: 악몽45, Average Similarity: 1.00
User: gurik, Average Similarity: 1.00
User: nom****, Average Similarity: 1.00
User: whl6237

In [ ]:
# 정렬된 결과를 데이터프레임으로 변환
total_similarity_df = pd.DataFrame(sorted_total_user_similarity, columns=['User', 'Average Similarity'])

# 정렬된 결과를 CSV 파일로 저장
output_file_path = 'total_similarity.csv'
total_similarity_df.to_csv(output_file_path, index=False)

In [ ]:
files.download(output_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>